<a href="https://colab.research.google.com/github/Sela80/s/blob/main/donn_es_bancaires_variable_de_sortie_cible_souhait_e_y_le_client_a_t_il_souscrit_un_d_p_t_terme_binaire_oui_non.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. Import des librairies & Chargement du dataset
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import joblib
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer # Import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix, RocCurveDisplay
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import ipywidgets as widgets
from IPython.display import display

In [ ]:
!pip install opendatasets --upgrade --quiet

In [ ]:
# Importation d'une Base de données Depuis Kaggle
import opendatasets as od
od.download('https://www.kaggle.com/datasets/prakharrathi25/banking-dataset-marketing-targets')

In [ ]:
# Chargement de notre Dataset
df=pd.read_csv('/content/banking-dataset-marketing-targets/test.csv', sep=';')

In [ ]:
#df[df['y'] == 'yes']

In [ ]:
df.head()


In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
df.columns

In [ ]:
df.duplicated().sum()

In [ ]:
df.describe().transpose()

In [ ]:
num_cols = df.select_dtypes(include=['int64', 'float64']).columns
cat_cols = df.select_dtypes(include=['object']).columns

In [ ]:
# 5) Numeric correlations heatmap (top features)
corr = df[num_cols].corr()
plt.figure(figsize=(8,6))
sns.heatmap(corr, annot=False, cmap='coolwarm', center=0)
plt.title('Corrélations entre variables numériques')
plt.tight_layout()
plt.show()

### Commentaire sur la matrice de corrélation numérique

La heatmap des corrélations entre les variables numériques montre les relations linéaires entre ces variables.

*   Les valeurs proches de 1 ou -1 indiquent une forte corrélation positive ou négative, respectivement.
*   Les valeurs proches de 0 indiquent une faible corrélation.

D'après cette heatmap :

*   Il n'y a pas de corrélations très fortes (proches de 1 ou -1) entre les paires de variables numériques.
*   Certaines corrélations modérées peuvent être observées, comme entre `pdays` et `previous` (corrélation positive, ce qui est attendu car `previous` compte le nombre de contacts avant `pdays`), et entre `day` et `campaign` (une légère corrélation positive).
*   La plupart des autres variables numériques montrent de faibles corrélations entre elles.

Cela suggère qu'il n'y a pas de problèmes majeurs de multicolinéarité entre les variables numériques dans ce dataset, ce qui est une bonne chose pour les modèles de régression linéaire ou logistique.

In [ ]:
# Plot target distribution
sns.countplot(x='y', data=df)
plt.title('Target distribution: y')
plt.xlabel('Subscribed to term deposit')
plt.ylabel('Count')
plt.show()


### Commentaire sur la distribution de la variable cible ('y')

Le graphique de distribution de la variable cible 'y' montre le nombre d'observations pour chaque catégorie : 'no' (n'ayant pas souscrit au dépôt à terme) et 'yes' (ayant souscrit au dépôt à terme).

D'après ce graphique, on observe une **forte disparité** entre les deux classes :

*   La grande majorité des clients n'ont **pas** souscrit au dépôt à terme ('no').
*   Un nombre beaucoup plus faible de clients ont **souscrit** au dépôt à terme ('yes').



In [ ]:
for col in cat_cols:
    plt.figure(figsize=(10, 6))
    sns.countplot(x=col, hue='y', data=df)
    plt.title(f'Distribution of {col} by Target (y)')
    plt.xlabel(col)
    plt.ylabel('Count')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

### Commentaire et conclusion sur les distributions des variables catégorielles par cible ('y')

Les graphiques de distribution des variables catégorielles par rapport à la variable cible 'y' (souscription à un dépôt à terme) révèlent des patterns intéressants sur les caractéristiques des clients qui ont tendance à souscrire.

En examinant les différents graphiques :

*   **Job:** Certains métiers comme "management", "blue-collar", et "technician" ont un grand nombre de clients, mais la proportion de clients ayant souscrit varie selon le métier. Les "students" et les "retired" semblent avoir une proportion plus élevée de souscription par rapport à leur nombre total dans l'échantillon.
*   **Marital:** Les clients "married" sont les plus nombreux, mais la proportion de souscription semble un peu plus élevée chez les "single" et les "divorced".
*   **Education:** Les clients avec une éducation "secondary" et "tertiary" sont les plus représentés. La proportion de souscription semble augmenter avec le niveau d'éducation, bien que les clients avec une éducation "unknown" aient aussi une proportion de souscription notable.
*   **Default:** La grande majorité des clients n'ont pas de défaut de crédit ('no'). La proportion de souscription est très faible pour les clients ayant un défaut ('yes').
*   **Housing:** Les clients ayant un prêt immobilier ('yes') et ceux n'en ayant pas ('no') sont assez nombreux. La proportion de souscription semble légèrement plus élevée chez les clients n'ayant pas de prêt immobilier.
*   **Loan:** Similaire à "housing", la plupart des clients n'ont pas de prêt personnel ('no'). La proportion de souscription est plus élevée chez les clients n'ayant pas de prêt personnel.
*   **Contact:** La méthode de contact "cellular" est la plus fréquente. Les contacts via "cellular" et "telephone" ont une proportion de souscription plus élevée que les contacts "unknown".
*   **Month:** Le mois de mai a le plus grand nombre de contacts, mais la proportion de souscription est relativement faible en mai par rapport à d'autres mois comme octobre, mars, décembre et septembre, qui, bien qu'ayant moins de contacts, montrent une proportion de souscription plus élevée.
*   **Poutcome:** Le résultat de la campagne précédente ('poutcome') a une forte influence. Les clients dont l'issue précédente était un succès ('success') ont une probabilité de souscription bien plus élevée lors de cette campagne.

**Conclusion sur les variables catégorielles:**

Plusieurs variables catégorielles montrent une distribution différente de la variable cible 'y', suggérant qu'elles sont informatives pour prédire la souscription à un dépôt à terme. `poutcome`, `contact`, `month`, `job`, `marital`, `education`, `housing`, et `loan` semblent être des prédicteurs potentiellement importants. La variable `default` semble moins discriminante car la grande majorité des clients n'ont pas de défaut et la souscription est rare pour ceux qui en ont un.

Ces visualisations confirment l'importance de ces variables catégorielles dans le modèle de prédiction et justifient l'utilisation de techniques de prétraitement comme l'One-Hot Encoding pour les inclure dans les modèles.

In [ ]:
for col in num_cols:
    plt.figure(figsize=(10, 6))
    sns.boxplot(x='y', y=col, data=df)
    plt.title(f'Distribution of {col} by Target (y)')
    plt.xlabel('Subscribed to term deposit')
    plt.ylabel(col)
    plt.tight_layout()
    plt.show()

### Commentaire et conclusion sur les distributions des variables numériques par cible ('y')

Les boxplots des variables numériques par rapport à la variable cible 'y' (souscription à un dépôt à terme) nous donnent un aperçu des différences de distribution de ces variables entre les clients qui ont souscrit et ceux qui ne l'ont pas fait.

En examinant les différents boxplots :

*   **age:** La distribution de l'âge semble assez similaire pour les deux groupes, bien qu'il y ait quelques valeurs extrêmes d'âges plus élevés dans le groupe "yes".
*   **balance:** La distribution du solde (`balance`) montre une grande variabilité, avec de nombreuses valeurs extrêmes. Bien que la médiane soit proche pour les deux groupes, il semble y avoir quelques soldes plus élevés dans le groupe "yes". Cependant, le chevauchement des distributions est important.
*   **day:** La distribution du jour du mois (`day`) où le contact a été effectué semble très similaire pour les deux groupes.
*   **duration:** La **durée du dernier contact** (`duration`) montre une différence notable. Les clients qui ont souscrit ('yes') ont tendance à avoir eu des contacts de plus longue durée que ceux qui n'ont pas souscrit ('no'). La médiane et le 3ème quartile sont significativement plus élevés pour le groupe "yes". Cette variable semble être un prédicteur important. Il est important de noter que cette variable n'est connue qu'APRÈS que le contact ait été effectué, ce qui peut la rendre moins utile pour la prédiction AVANT le contact.
*   **campaign:** Le nombre de contacts effectués pendant cette campagne (`campaign`) semble légèrement plus élevé pour le groupe "no", avec de nombreuses valeurs extrêmes. Les clients qui souscrivent ont tendance à avoir eu moins de contacts pendant la campagne actuelle.
*   **pdays:** Le nombre de jours depuis le dernier contact de la campagne précédente (`pdays`) montre une différence. Pour les clients qui n'ont pas souscrit, la majorité ont une valeur de -1 (pas de contact précédent). Pour ceux qui ont souscrit, il y a une distribution de valeurs positives, indiquant un contact récent lors d'une campagne précédente. Cette variable semble informative.
*   **previous:** Le nombre de contacts effectués avant cette campagne (`previous`) montre que les clients qui ont souscrit ont tendance à avoir eu plus de contacts précédents réussis (liés à `poutcome`).

**Conclusion sur les variables numériques:**

Parmi les variables numériques, la **durée du dernier contact (`duration`)** et le nombre de **jours depuis le dernier contact de la campagne précédente (`pdays`)** semblent être les prédicteurs les plus discriminants pour la souscription à un dépôt à terme. La variable `campaign` montre également une certaine différence. Les variables `age`, `balance`, et `day` semblent avoir une influence moins marquée sur la souscription, bien que `balance` présente des valeurs extrêmes potentiellement intéressantes.

Il est crucial de considérer l'interprétation de la variable `duration` : bien qu'elle soit très prédictive, elle est une conséquence du contact, pas une caractéristique préalable du client. Son utilisation dans un modèle de prédiction pour cibler les clients *avant* le contact doit être faite avec prudence. Cependant, elle est très utile pour évaluer le succès d'une campagne après coup. Les autres variables numériques (`age`, `balance`, `day`, `campaign`, `pdays`, `previous`) représentent des caractéristiques du client ou de l'historique des campagnes qui sont disponibles avant un nouveau contact.

In [ ]:
# Définition des colonnes
cat_cols = ["job", "marital", "education", "default", "housing", "loan",
            "contact", "month", "poutcome"]
num_cols = ["age", "balance", "day", "duration", "campaign", "pdays", "previous"]


In [ ]:

X = df.drop(columns=["y"])
y = (df["y"] == "yes").astype(int)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
# 3) Prétraitement
# ============================
preprocess = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=False), cat_cols),
        ("num", StandardScaler(), num_cols),
    ]
)


In [ ]:
log_reg = Pipeline(steps=[
    ('prep', preprocess),
    ('clf', LogisticRegression(max_iter=1000, class_weight='balanced'))
])

rf = Pipeline(steps=[
    ('prep', preprocess),
    ('clf', RandomForestClassifier(n_estimators=300, class_weight='balanced_subsample', random_state=42))
])

In [ ]:

# Validation croisée
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
log_auc = cross_val_score(log_reg, X_train, y_train, cv=cv, scoring='roc_auc')
rf_auc = cross_val_score(rf, X_train, y_train, cv=cv, scoring='roc_auc')

print(f'Logistic Regression CV AUC: {log_auc.mean():.4f} ± {log_auc.std():.4f}')
print(f'Random Forest CV AUC: {rf_auc.mean():.4f} ± {rf_auc.std():.4f}')


### Commentaire sur les Résultats de Validation Croisée

Les résultats de la validation croisée (avec 5 folds stratifiés) pour les deux modèles sont les suivants, basés sur la métrique AUC (Area Under the ROC Curve) :

*   **Logistic Regression CV AUC:** 0.8939 ± 0.0199
*   **Random Forest CV AUC:** 0.9076 ± 0.0139

**Interprétation:**

*   L'**AUC moyenne** sur les 5 folds est légèrement plus élevée pour le modèle **Random Forest** (0.9076) par rapport à la **Logistic Regression** (0.8939). Cela suggère que, en moyenne sur différents sous-ensembles des données d'entraînement, le modèle Random Forest a une meilleure capacité globale à distinguer les clients qui souscrivent de ceux qui ne souscrivent pas.
*   L'**écart-type** de l'AUC est légèrement plus faible pour le modèle **Random Forest** (0.0139) que pour la **Logistic Regression** (0.0199). Un écart-type plus faible indique que la performance du modèle est plus stable à travers les différents folds de la validation croisée. Cela suggère que le modèle Random Forest est un peu plus robuste et moins sensible aux variations dans les données d'entraînement par rapport à la régression logistique.

**Conclusion de la Validation Croisée:**

Les résultats de la validation croisée suggèrent que le modèle de **Random Forest** offre une performance légèrement meilleure et plus stable en termes d'AUC sur l'ensemble d'entraînement. Cela renforce l'idée que la forêt aléatoire pourrait être un modèle légèrement plus performant pour ce problème de classification, en particulier si l'on considère l'AUC comme métrique principale d'évaluation. Cependant, les deux modèles montrent de bonnes performances avec des AUC supérieures à 0.89, ce qui indique qu'ils sont capables d'apprendre des patterns pertinents dans les données.

In [ ]:

# Fit final
log_reg.fit(X_train, y_train)
rf.fit(X_train, y_train)


In [ ]:
# Évaluation
def print_metrics(name, model, X, y):
    proba = model.predict_proba(X)[:,1]
    pred = (proba >= 0.5).astype(int)
    acc = accuracy_score(y, pred)
    auc = roc_auc_score(y, proba)
    print(f"\n{name} - Accuracy: {acc:.3f}, AUC: {auc:.3f}")
    print(classification_report(y, pred, digits=3))
    print("Confusion matrix:\n", confusion_matrix(y, pred))

print_metrics("Logistic Regression", log_reg, X_test, y_test)
print_metrics("Random Forest", rf, X_test, y_test)


### Commentaire sur les résultats de l'Évaluation

Les métriques d'évaluation sur l'ensemble de test fournissent une image plus complète des performances de chaque modèle dans un scénario réel.

*   **Logistic Regression:**
    *   Accuracy (Précision globale): 0.823. Le modèle prédit correctement 82.3% des cas.
    *   AUC: 0.891. Bonne capacité à distinguer les classes.
    *   Classification Report:
        *   Precision (Classe 'yes', 1): 0.372. Sur toutes les fois où le modèle a prédit 'yes', il avait raison dans 37.2% des cas.
        *   Recall (Classe 'yes', 1): 0.779. Le modèle a identifié 77.9% de tous les clients qui ont réellement souscrit ('yes'). C'est un bon rappel, ce qui signifie qu'il manque peu de vrais positifs.
        *   F1-score (Classe 'yes', 1): 0.503. Moyenne harmonique de la précision et du rappel pour la classe positive.
    *   Confusion Matrix:
        *   True Negatives (TN): 664 (Correctly predicted 'no')
        *   False Positives (FP): 137 (Incorrectly predicted 'yes', they were 'no') - Erreur de type I
        *   False Negatives (FN): 23 (Incorrectly predicted 'no', they were 'yes') - Erreur de type II
        *   True Positives (TP): 81 (Correctly predicted 'yes')

*   **Random Forest:**
    *   Accuracy: 0.891. Précision globale plus élevée que la régression logistique.
    *   AUC: 0.912. Légèrement meilleure capacité à distinguer les classes que la régression logistique.
    *   Classification Report:
        *   Precision (Classe 'yes', 1): 0.581. Sur toutes les fois où le modèle a prédit 'yes', il avait raison dans 58.1% des cas. Meilleure précision que la régression logistique.
        *   Recall (Classe 'yes', 1): 0.173. Le modèle a identifié seulement 17.3% de tous les clients qui ont réellement souscrit ('yes'). C'est un rappel très faible, ce qui signifie qu'il manque beaucoup de vrais positifs (il génère beaucoup de faux négatifs).
        *   F1-score (Classe 'yes', 1): 0.267. F1-score beaucoup plus faible que la régression logistique pour la classe positive.
    *   Confusion Matrix:
        *   TN: 788
        *   FP: 13 (Beaucoup moins de faux positifs que la régression logistique)
        *   FN: 86 (Beaucoup plus de faux négatifs que la régression logistique)
        *   TP: 18

**Conclusion de l'Évaluation:**

Les résultats sur l'ensemble de test confirment que la **Random Forest** a une meilleure performance globale en termes d'Accuracy et d'AUC. Cependant, pour le problème spécifique de prédire les souscriptions (la classe minoritaire 'yes'), la **Logistic Regression**, malgré une Accuracy et une AUC légèrement inférieures, démontre un **rappel (Recall) bien supérieur**.

*   Si l'objectif est de maximiser le nombre de clients potentiels identifiés (même au prix de contacter des personnes qui ne souscriront pas), la **régression logistique** est plus adaptée en raison de son **rappel élevé** (77.9% des vrais positifs capturés). Elle minimise les erreurs de Type II (faux négatifs).
*   Si l'objectif est de minimiser les contacts inutiles et de s'assurer que les clients contactés ont une forte probabilité de souscrire, la **forêt aléatoire** est préférable en raison de sa **précision élevée** (58.1% des prédictions positives sont correctes). Elle minimise les erreurs de Type I (faux positifs).

Le choix final du modèle dépendra des coûts associés aux faux positifs (contacter un client qui ne souscrit pas) et aux faux négatifs (ne pas contacter un client qui aurait souscrit). Dans un contexte marketing où l'identification de potentiels clients est primordiale, un rappel élevé est souvent plus souhaitable, suggérant que la régression logistique pourrait être plus pertinente malgré un plus grand nombre de faux positifs. Cependant, si les coûts de contact sont élevés, la forêt aléatoire pourrait être préférée.

In [ ]:

# Courbes ROC
RocCurveDisplay.from_estimator(log_reg, X_test, y_test, name='LogReg')
RocCurveDisplay.from_estimator(rf, X_test, y_test, name='RandomForest')
plt.title('ROC curves on Test')
plt.show()


In [ ]:
# 5) Sauvegarde modèles
# ============================
joblib.dump(log_reg, 'model_logreg_bank.joblib')
joblib.dump(rf, 'model_rf_bank.joblib')
print("Modèles sauvegardés.")


In [ ]:

# 6) Widget interactif
# ============================
options = {col: sorted(df[col].dropna().unique().astype(str)) for col in cat_cols}

# Widgets numériques
w_age = widgets.BoundedIntText(value=35, min=17, max=100, description='age')
w_balance = widgets.IntText(value=0, description='balance')
w_day = widgets.BoundedIntText(value=5, min=1, max=31, description='day')
w_duration = widgets.IntText(value=120, description='duration')
w_campaign = widgets.BoundedIntText(value=1, min=1, max=100, description='campaign')
w_pdays = widgets.IntText(value=-1, description='pdays')
w_previous = widgets.BoundedIntText(value=0, min=0, max=100, description='previous')

# Widgets catégoriels
w_job = widgets.Dropdown(options=options['job'], description='job')
w_marital = widgets.Dropdown(options=options['marital'], description='marital')
w_education = widgets.Dropdown(options=options['education'], description='education')
w_default = widgets.Dropdown(options=options['default'], description='default')
w_housing = widgets.Dropdown(options=options['housing'], description='housing')
w_loan = widgets.Dropdown(options=options['loan'], description='loan')
w_contact = widgets.Dropdown(options=options['contact'], description='contact')
w_month = widgets.Dropdown(options=options['month'], description='month')
w_poutcome = widgets.Dropdown(options=options['poutcome'], description='poutcome')

btn = widgets.Button(description='Prédire', button_style='primary')
out = widgets.Output()

def on_click_predict(b):
    out.clear_output()
    row = {
        'age': w_age.value, 'job': w_job.value, 'marital': w_marital.value,
        'education': w_education.value, 'default': w_default.value,
        'balance': w_balance.value, 'housing': w_housing.value, 'loan': w_loan.value,
        'contact': w_contact.value, 'day': w_day.value, 'month': w_month.value,
        'duration': w_duration.value, 'campaign': w_campaign.value,
        'pdays': w_pdays.value, 'previous': w_previous.value, 'poutcome': w_poutcome.value
    }
    X_single = pd.DataFrame([row])
    try:
        proba = log_reg.predict_proba(X_single)[:,1][0]
        pred = log_reg.predict(X_single)[0]
        label = "oui" if pred == 1 else "non"
        with out:
            print(f"Résultat: {label}")
            print(f"Probabilité (oui): {proba*100:.2f} %")
    except Exception as e:
        with out:
            print("Erreur pendant la prédiction:", e)

btn.on_click(on_click_predict)

ui = widgets.HBox([
    widgets.VBox([w_age, w_balance, w_day, w_duration, w_campaign, w_pdays, w_previous]),
    widgets.VBox([w_job, w_marital, w_education, w_default, w_housing, w_loan, w_contact, w_month, w_poutcome])
])

display(widgets.VBox([ui, btn, out]))
print("Widget prêt. Choisissez des valeurs puis cliquez sur Prédire.")
